# Import Libraries

In [1]:
from swarmintelligence import GeneticAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/GA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_gens': 100, 'mr': 0.2, 'cr': 0.6}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    gwo_g = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    gwo_b = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
925.8012639780487  |  2436.0842345679075  |  1766.5713325872546
926.8429316685265  |  2395.8394485376675  |  1784.9314857284867
894.5434898874109  |  2426.1918070515667  |  1785.3243200520578
922.4758900390822  |  2423.639911388777  |  1776.4772588408973
929.440098653832  |  2403.85962770689  |  1783.0708974767272
906.3958082539227  |  2431.7446398020747  |  1755.4011287895332
910.5242854163517  |  2428.2207929098286  |  1784.580071190192
915.4568712539175  |  2409.903266389748  |  1724.9122544291756
884.448051542194  |  2423.0848525058595  |  1786.0813262042361
928.5564414276095  |  2418.5056261447053  |  1780.738164998902
930.6342523318668  |  2410.9549679965803  |  1787.645984455087
922.657084777582  |  2375.333544926695  |  1783.7514753286011
924.6535842403083  |  2382.792200614559  |  1779.8909335858746
924.0893658762285  |  2410.230950623852  |  1777.2095521818014
913.5409976373123  |  2400.6687212586926  |  1764.0467205875811
924.2453143528767

976.4520065411689  |  2376.7453707281693  |  2180.750043598501
1001.8944097724988  |  2343.6908196446084  |  2148.0859525592714
1004.7658946943336  |  2382.786009584833  |  2136.6842347707206
976.1048743608198  |  2384.386901393847  |  2063.2088093170028
964.9786096430039  |  2390.34621187027  |  2062.1395497584185
991.229475553889  |  2334.1228592888156  |  2173.725250108231
995.2843274890404  |  2197.5738821447735  |  2172.6599454492525
983.285476209634  |  2387.694862293263  |  2119.513818956575
997.2786057399302  |  2347.7522743481536  |  2104.508848244151
977.7172082748176  |  2290.3718483548028  |  2119.513818956575
977.6251123935431  |  2365.527862080665  |  2179.356728980569
985.3280173373763  |  2352.747438213237  |  2163.2470334386
986.7037779792813  |  2380.761946051759  |  2163.1454854905173
994.0163999757494  |  2385.8989312385374  |  2181.758352748785
969.3683218916716  |  2356.804085108708  |  2165.194672317649
1000.5186038846792  |  2388.7577067813872  |  2123.624979665

2934.4403981999058  |  2912.383624928084  |  2940.7487388451796
2857.5322826160454  |  2911.1682083991864  |  2850.4118205741306
2936.005408229921  |  2900.7987608380226  |  2927.7823540343347
2902.4324279606108  |  2879.8940283385305  |  2921.045270970181
2946.7758196466157  |  2900.610701894222  |  2936.580829848667
2944.9560484369395  |  2914.5957232653755  |  2947.0383490755203
2931.4263893981765  |  2924.1209714153456  |  2945.955455751833
2943.1877961505525  |  2918.7193072943837  |  2848.460457314083
2882.079794594749  |  2934.199070581145  |  2946.101610024072
2870.9420562346736  |  2909.912852084167  |  2918.069338207195
2933.22851243446  |  2915.625569676007  |  2898.5745135639163
2939.1045736352453  |  2826.386315012622  |  2946.645177162376
2922.1211755292406  |  2942.411765596952  |  2917.260070538715
2943.271856701946  |  2897.9404375841523  |  2945.367568824766
2940.440962103831  |  2901.378698307657  |  2905.0395600163815
2945.6995307403713  |  2934.2277493890483  |  29

3345.05326950387  |  1979.0516747304935  |  2585.194628216192
3358.9034711276445  |  1998.4311821593062  |  2552.3576656724763
3337.671962801176  |  1977.5159821179736  |  2613.375024597906
3348.467769684905  |  1980.743357878238  |  2619.9580488421707
3305.0356342586247  |  1986.383500186274  |  2630.4008922077483
3343.47609924007  |  1999.2957245779605  |  2496.2942735380193
3317.3588721914907  |  1994.729801638427  |  2606.763488079053
3336.7536786290743  |  1993.3276263314863  |  2631.710374523379
3276.1946248347112  |  2001.6140111808572  |  2604.027641402761
3345.4586818139114  |  1999.435483346164  |  2571.9455608400685
3288.9348447971734  |  1968.2298429584866  |  2602.1169965864647
3215.7454196394065  |  1995.1943843603076  |  2589.9895768330234
3350.6852217219457  |  1954.1130126100275  |  2621.0275981164286
3354.6357606678057  |  1986.488754126059  |  2616.7124123122558
3323.2033599874726  |  1993.6510147170363  |  2633.6669873456603
3359.0721170082556  |  1931.8857331209474

1630.4964355064471  |  5189.969861646011  |  1720.7754973850192
1655.229908758901  |  5192.470832653759  |  1721.8155925852898
1621.7312209323295  |  5197.732790338499  |  1712.8506175002792
1631.188814229128  |  5174.541688112042  |  1750.6432352298484
1649.5068412842309  |  5159.477946588973  |  1754.4636559804248
1606.4629249134973  |  5202.461953578297  |  1752.19946347823
1643.7266684807778  |  5172.866388129785  |  1757.2119760446271
1620.7938430015633  |  5189.303979979395  |  1715.2315998884883
1641.3580133692835  |  5189.67598480048  |  1724.256912159046
1558.274844791868  |  5194.0808049693915  |  1756.1043019362214
1614.1021827368456  |  5202.725704995132  |  1753.1804377181772
1631.640411809814  |  5179.623985288114  |  1743.676690834682
1636.3058510856006  |  5144.971371256788  |  1719.6600790014295
1644.0738654889528  |  5158.774147345157  |  1751.6621015216956
1654.7387095116621  |  5200.533827778781  |  1754.6067768993912
1651.412150021431  |  5200.706916177355  |  1757

5627.759433092666  |  5627.736668874561  |  1572.0313171386915
5599.65184022036  |  5607.507348714268  |  1573.8038248646064
5638.140820643906  |  5633.11923894273  |  1583.6610416720732
5532.757506751915  |  5605.930485962212  |  1558.555644111215
5636.444539759211  |  5501.171513321569  |  1585.3898811601707
5627.113870713366  |  5621.352890233272  |  1538.3168275638095
5608.463515830762  |  5616.82360138729  |  1513.343162647147
5600.672527575143  |  5633.669627835332  |  1576.6171872851996
5595.603526002895  |  5632.677638060263  |  1556.6747206382238
5537.607785278052  |  5448.479894190662  |  1586.950647543145
5629.416990290312  |  5627.908446806305  |  1551.9613960374336
5580.895271331331  |  5606.7438812275595  |  1573.7253448093575
5611.597557598292  |  5627.468748604257  |  1569.6686957507452
5637.575255228054  |  5572.450981892621  |  1587.436497888975
5636.38070557062  |  5608.165928752605  |  1583.0517403629474
5617.267004888983  |  5570.993378690194  |  1580.296206184681


## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[120, 173]","[100, 166]","[120, 181]",917.873218,2412.739608,1773.192571,...,"[[120, 255, 255, 255, 255, 255, 255, 255, 255,...","[925.8012639780487, 926.8429316685265, 894.543...","[2436.0842345679075, 2395.8394485376675, 2426....","[1766.5713325872546, 1784.9314857284867, 1785....","[3.4277639389038086, 3.35463809967041, 3.06435...","[2231.5466702779136, 2152.6948280334473, 2446....","[47.2392492560785, 46.39714245547292, 49.46323...","[14.644743868178612, 14.800978934665736, 14.24...","[0.7609855497783665, 0.7553184442100545, 0.775...","[0.9409319823764326, 0.9415723428226487, 0.937..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[99, 146, 189]","[71, 140, 184]","[120, 168, 194]",956.869350,2505.838081,1831.534828,...,"[[120, 255, 255, 255, 255, 255, 255, 255, 255,...","[963.0087782034354, 964.9350574701936, 953.629...","[2520.6358920812286, 2503.2961955482624, 2468....","[1837.9424405128416, 1816.4460020397603, 1814....","[4.1803672313690186, 4.800585746765137, 4.5522...","[1944.8404299418132, 1891.897986094157, 1788.8...","[44.10034500932859, 43.49595367495874, 42.2944...","[15.241963866913487, 15.361826459643106, 15.60...","[0.7730756654475538, 0.7582899465458581, 0.753...","[0.9508879152070256, 0.9495400719342939, 0.953..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[83, 125, 160, 195]","[68, 118, 159, 192]","[105, 146, 176, 200]",973.081497,2554.087508,1868.261662,...,"[[105, 255, 200, 255, 255, 255, 200, 200, 200,...","[945.5175848763596, 983.7406776724557, 965.337...","[2542.282155577804, 2554.5218078329826, 2532.7...","[1886.8789781144858, 1871.9925534633855, 1885....","[3.9033243656158447, 3.010204315185547, 3.0249...","[1873.0610516866047, 1542.937836964925, 1352.7...","[43.27887535145298, 39.28024741476211, 36.7793...","[15.405284276045334, 16.2473193163363, 16.8187...","[0.7847353492137396, 0.7435518201974238, 0.778...","[0.9556044442370601, 0.9613112816955711, 0.963..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[77, 112, 144, 175, 201]","[55, 99, 142, 173, 204]","[91, 131, 161, 182, 202]",987.793577,2581.614816,1893.596931,...,"[[131, 255, 202, 255, 255, 202, 202, 202, 202,...","[990.995230488468, 989.444069172579, 986.82684...","[2595.5438430502995, 2610.4415178376116, 2562....","[1894.9121851838504, 1898.2655670364925, 1908....","[3.982512950897217, 3.9038801193237305, 3.4379...","[1508.376013437907, 1199.673188527425, 1522.43...","[38.83781679546247, 34.636298712873824, 39.018...","[16.34570743335856, 17.340174079451312, 16.305...","[0.7754945236764547, 0.7467919847390311, 0.751...","[0.9639596463263658, 0.9694992418442777, 0.962..."
4,Lena.png,2,mMasi_entropy,max,"[134, 188]","[74, 141]","[97, 140]",990.974327,2356.674172,2144.012635,...,"[[140, 140, 140, 140, 140, 140, 140, 140, 140,...","[976.4520065411689, 1001.8944097724988, 1004.7...","[2376.7453707281693, 2343.6908196446084, 2382....","[2180.750043598501, 2148.0859525592714, 2136.6...","[3.6944851875305176, 3.7632904052734375, 3.192...","[1922.382480621338, 2596.4025675455728, 2129.0...","[43.84498238819737, 50.95490719788991, 46.1421...","[15.292405609393935, 13.987083309409753, 14.84...","[0.628040241378804, 0.6267210566305587, 0.6155...","[0.8930936696064565, 0.8867984338526765, 0.890..."
5,Lena.png,3,mMasi_entropy,max,"[126, 170, 210]","[62, 105, 162]","[84, 117, 155]",1043.914397,2524.988354,2235.983748,...,"[[155, 155, 155, 155, 155, 117, 155, 155, 155,...","[1049.2930623588181, 1054.346459498789, 1006.6...","[2589.3665297325924, 2569.3002358383364, 2542....","[2222.564819590759, 2242.8100885530075, 2255.9...","[3.120368719100952, 3.003732204437256, 2.98256...","[1109.2494710286458, 1390.5949325561523, 1214....","[33.305397025536955, 37.29068157805851, 34.847...","[17.6805113

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/GA/GA_masi_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/GA/GA_masi_RGB_dict.pkl', 'wb'))